## Conhecendo o DeltaLake

In [ ]:
#Instalando as dependencias
!pip install -r requirements.txt -q

In [1]:
import pyspark

spark = pyspark.sql.SparkSession.builder.appName("JobDeltaLake") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.8.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.databricks.delta.schema.autoMerge.enabled","true") \
    .config("spark.databricks.delta.autoOptimize.optimizeWrite","true") \
    .config("spark.databricks.delta.optimizeWrite.enabled","true") \
    .config("spark.databricks.delta.vacuum.parallelDelete.enabled","true") \
    .getOrCreate()

from delta.tables import *
from pyspark.sql.functions import *

from datetime import date

In [2]:
spark

Criando os paths

In [3]:
path_input = 'input/'

In [4]:
path_output = 'output/'

In [5]:
!rm -rf output/

In [6]:
!rm -rf input/

### Criando os datasets

In [7]:
df_1 = spark.createDataFrame(
    [
       (1, date.today(), 'Primeira linha dataframe'),
       (2, date.today(), 'Segunda linha dataframe'),
    ],
        ['id','data', 'texto']
)

df_1.show()

+---+----------+--------------------+
| id|      data|               texto|
+---+----------+--------------------+
|  1|2021-05-30|Primeira linha da...|
|  2|2021-05-30|Segunda linha dat...|
+---+----------+--------------------+



Vamos salvar o arquivo em formato parquet, para emular a leitura de um bucket no Storage com arquivos .parquet.

In [8]:
#Escrevendo o arquivo no formato parquet
df_1.write.format("parquet").save(path_input)

In [10]:
#Fazendo a leitura
df_1 = spark.read.format("parquet").option("inferSchema", "false").load(path_input)
df_1.show(truncate=False)

+---+----------+------------------------+
|id |data      |texto                   |
+---+----------+------------------------+
|1  |2021-05-30|Primeira linha dataframe|
|2  |2021-05-30|Segunda linha dataframe |
+---+----------+------------------------+



In [11]:
#Escrevendo o dataframe em formato delta
df_1.write.format("delta").save(path_output)

### Lendo os dados

In [12]:
#Verificando se o arquivo foi salvo no formato delta
df = spark.read.format("delta").load(path_output)
df.show(truncate=False)

+---+----------+------------------------+
|id |data      |texto                   |
+---+----------+------------------------+
|1  |2021-05-30|Primeira linha dataframe|
|2  |2021-05-30|Segunda linha dataframe |
+---+----------+------------------------+



In [14]:
#Vamos criar um segundo dataframe
df_2 = spark.createDataFrame(
    [
        (1, date.today(), 'Primeira linha dataframe'),
        (2, date.today(), 'Segunda linha att dataframe'),
        (3, date.today(), 'Terceira linha dataframe'),

    ],
        ['id','data', 'texto']
)

df_2.show(truncate=False)

+---+----------+---------------------------+
|id |data      |texto                      |
+---+----------+---------------------------+
|1  |2021-05-30|Primeira linha dataframe   |
|2  |2021-05-30|Segunda linha att dataframe|
|3  |2021-05-30|Terceira linha dataframe   |
+---+----------+---------------------------+



In [15]:
#Escrevendo em tabela
table = DeltaTable.forPath(spark, path_output)

In [16]:
table.toDF().show(truncate=False)

+---+----------+------------------------+
|id |data      |texto                   |
+---+----------+------------------------+
|1  |2021-05-30|Primeira linha dataframe|
|2  |2021-05-30|Segunda linha dataframe |
+---+----------+------------------------+



In [17]:
#Fazendo o merge dos arquivos
table.alias("persisteddata") .merge( \
   df_2.alias("newdata"), \
    "persisteddata.id = newdata.id") \
.whenMatchedUpdateAll() \
.whenNotMatchedInsertAll() \
.execute()

In [18]:
table.toDF().show(truncate=False)

+---+----------+---------------------------+
|id |data      |texto                      |
+---+----------+---------------------------+
|2  |2021-05-30|Segunda linha att dataframe|
|3  |2021-05-30|Terceira linha dataframe   |
|1  |2021-05-30|Primeira linha dataframe   |
+---+----------+---------------------------+



In [24]:
table.vacuum()

DataFrame[]

In [25]:
#Verificando o Schema
table.toDF().printSchema()

root
 |-- id: long (nullable = true)
 |-- data: date (nullable = true)
 |-- texto: string (nullable = true)



## Verificando as versões

In [ ]:
df = spark.read.format("delta").option("versionAsOf", 0).load(path_output)
df.show(truncate=False)

In [ ]:
df = spark.read.format("delta").option("versionAsOf", 1).load(path_output)
df.show(truncate=False)